In [1]:
from typing import List, Optional

from openai import OpenAI
from dotenv import load_dotenv
import os
import glob
from docx import Document
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import json
import io
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('max_colwidth', None)

load_dotenv()

True

In [2]:
def load_doc(path: str, add_tables: bool = False):
    doc = Document(path)
    full_text = []
    for paragraph in doc.paragraphs:
        if len(paragraph.text) > 0:
            full_text.append(paragraph.text)
    
    text = "\n".join(full_text)
    if add_tables:
        tables_str = []
        for table_index, table in enumerate(doc.tables):
            table_str = f"Таблица {table_index + 1}"
            table_data = []
            for row in table.rows:
                row_data = [cell.text for cell in row.cells]
                table_data.append(row_data)
            markup_table = pd.DataFrame(np.vstack(table_data)).to_markdown(index=False)
            table_str += "\n" + markup_table
            tables_str.append(table_str)
        tables_full_str = "\n".join(tables_str)
        text = text + "\n" + tables_full_str
    return text

In [75]:
def find_info(tech_task: str, query: str, examples: Optional[List] = None, query_desc: Optional[str] = None):
    question = f"Какие {query} должны быть у специалиста из должностной инструкции?"
    prompt = f"Должностная инструкция:\n{tech_task}\nВопрос: {question}\nОтвет:"
    system_prompt = f"Вы эксперт в найме персонала. Необходимо извлечь необходимые {query} из должностной инструкции, "\
                    f"как отдельное поле в json с ключом '{query}'. "\
                    f"Не нужно придумывать того, чего нет в должностной инструкции."
    if query_desc is not None:
        system_prompt += f"\n{query_desc}"
    if examples is not None:
        examples_str = '\n'.join(examples)
        system_prompt = system_prompt + f" Примеры {query}:\n{examples_str}"
        
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format={ "type": "json_object"}
    )
    return completion.choices[0].message.content

In [4]:
def get_embedding(text, model="text-embedding-3-large"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_emb_list(texts_list, model="text-embedding-3-large"):
    res = []
    for text in tqdm(texts_list):
        emb = np.array(get_embedding(text=text))
        res.append(emb)
    return np.array(res)

def find_match_cosine(query, emb_ref, emb_query, all_competencies, comp_query):
    similarities = cosine_similarity(emb_query, emb_ref)
    idices_most_similar = similarities.argmax(axis = 1)
    similarities = similarities.max(axis = 1).round(3)
    most_similar = np.array(all_competencies)[idices_most_similar]
    df_sim = pd.DataFrame(
        np.vstack([comp_query, most_similar, similarities]).T,
        columns = [f"Найденные {query}", f"Существующие {query}", "cosine_sim"]
    )
    return df_sim

In [5]:
client = OpenAI(api_key=os.getenv("OPENAI_TOKEN"))

## Load data

In [40]:
df = pd.read_excel("./data/karti competencii_marketing.xlsx", skiprows = [0])

In [41]:
queries_mapping = {
    "знания": "Unnamed: 8",
    "должности": "Unnamed: 5",
    "навыки": "Unnamed: 11"
}

In [42]:
ref_characteristics = {
    key : list(filter(lambda x: type(x) == str, df[queries_mapping[key]].dropna().unique()))
    for key in queries_mapping
}

In [57]:
ref_characteristics["Soft Skills"] = None

In [43]:
list_queries = list(queries_mapping.keys()) + ["Soft Skills"]

In [44]:
list_queries

['знания', 'должности', 'навыки', 'софт-скиллы']

In [45]:
query_descs = {
    "знания": "Знания - это тот набор знаний, который кандидат должен иметь.",
    "должности": "Должность - это позиция, вакансия, на которую претендует кандидат.",
    "навыки": "Навыки - умения, которыми кандидат должен обладать.",
    "софт-скиллы": "Soft Skills — это совокупность умений, которые показывают, каким характером обладает человек "\
                   "и как успешно он взаимодействует с другими людьми.",
}

In [46]:
ref_characteristics.keys()

dict_keys(['знания', 'должности', 'навыки'])

In [48]:
ref_characteristics_embs = {key : get_emb_list(ref_characteristics[key]) for key in ref_characteristics}

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 325/325 [02:31<00:00,  2.15it/s]


## Маркетолог

In [30]:
text_task = load_doc("./data/долж_инструкция_маркетолог.docx")

In [58]:
found_characteristics = {}
dfs_match = {}
for query in tqdm(query_descs):
    characteristics_list_str = find_info(
        text_task,
        query,
        examples=ref_characteristics[query],
        query_desc=query_descs[query]
    )
    characteristics_list = json.loads(characteristics_list_str)[query]
    found_characteristics[query] = characteristics_list
    if query != "софт-скиллы":
        characteristics_embs = get_emb_list(characteristics_list)
        df_sim_query = find_match_cosine(
            query,
            ref_characteristics_embs[query],
            characteristics_embs,
            ref_characteristics[query],
            characteristics_list,
        )
        dfs_match[query] = df_sim_query

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:45<00:00, 11.40s/it]


In [61]:
for key in dfs_match:
    dfs_match[key].to_csv(f"./save/match_results_{key}.csv", index = False)

In [62]:
dfs_match["должности"]

,Найденные должности,Существующие должности,cosine_sim
0,Маркетолог,Маркетолог,1.0


In [100]:
desc = "Soft Skills — это совокупность умений, которые показывают, каким характером обладает человек "\
"и как успешно он взаимодействует с другими людьми."
characteristics_list_str = find_info(
    text_task,
    "Soft Skills",
    examples=None,
    query_desc=desc,
)

In [101]:
characteristics_list = json.loads(characteristics_list_str)["Soft Skills"]
characteristics_list

['Системное мышление',
 'Стратегическое мышление',
 'Добросовестность',
 'Соблюдение трудовой дисциплины',
 'Организационные способности',
 'Способность к анализу и контролю',
 'Умение разрабатывать и реализовывать стратегии',
 'Навыки взаимодействия с сотрудниками и руководством',
 'Способность работать в команде',
 'Навыки ведения деловых коммуникаций',
 'Умение проводить эффективные мониторинги и анализы']

In [102]:
df_ = pd.read_csv("./save/match_results_знания_ext.csv")

In [104]:
df_["Соответствие"].value_counts(normalize=True).round(3).to_frame()

,proportion
Соответствие,
Идеально,0.632
Хорошо,0.263
Норм,0.053
Не совпадает,0.053


In [105]:
df_ = pd.read_csv("./save/match_results_навыки_ext.csv")

In [106]:
df_["Соответствие"].value_counts(normalize=True).round(3).to_frame()

,proportion
Соответствие,
Хорошо,0.529
Идеально,0.324
Норм,0.088
Не соответствует,0.059


## Директор по маркетингу

In [107]:
text_task = load_doc("./data/долж_инструкция_директор_маркетинг.docx")

In [64]:
found_characteristics_director = {}
dfs_match_director = {}
for query in tqdm(query_descs):
    characteristics_list_str = find_info(
        text_task,
        query,
        examples=ref_characteristics[query],
        query_desc=query_descs[query]
    )
    characteristics_list = json.loads(characteristics_list_str)[query]
    found_characteristics[query] = characteristics_list
    if query != "софт-скиллы":
        characteristics_embs = get_emb_list(characteristics_list)
        df_sim_query = find_match_cosine(
            query,
            ref_characteristics_embs[query],
            characteristics_embs,
            ref_characteristics[query],
            characteristics_list,
        )
        dfs_match[query] = df_sim_query

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:46<00:00, 11.65s/it]


In [65]:
for key in dfs_match:
    dfs_match[key].to_csv(f"./save/match_results_director_{key}.csv", index = False)

In [66]:
dfs_match["должности"]

,Найденные должности,Существующие должности,cosine_sim
0,Директор по маркетингу,Директор по маркетингу,1.0


In [108]:
desc = "Soft Skills — это совокупность умений, которые показывают, каким характером обладает человек "\
"и как успешно он взаимодействует с другими людьми."
characteristics_list_str = find_info(
    text_task,
    "Soft Skills",
    examples=None,
    query_desc=desc,
)

In [109]:
characteristics_list = json.loads(characteristics_list_str)["Soft Skills"]
characteristics_list

['Системное мышление',
 'Стратегическое мышление',
 'Этика делового общения',
 'Коммуникационные навыки',
 'Организационные способности',
 'Умение планировать и координировать',
 'Способность к аналитическому мышлению',
 'Навыки взаимодействия с различными подразделениями']

In [2]:
df_ = pd.read_csv("./save/match_results_director_знания_ext.csv")

In [3]:
df_["Соответствие"].value_counts(normalize=True).round(3).to_frame()

,proportion
Соответствие,
Идеально,0.857
Хорошо,0.143


In [4]:
df_ = pd.read_csv("./save/match_results_director_навыки_ext.csv")

In [5]:
df_["Соответствие"].value_counts(normalize=True).round(3).to_frame()

,proportion
Соответствие,
Идеально,0.519
Хорошо,0.333
Норм,0.111
Не соответствует,0.037
